In [ ]:
p1="/content/drive/Othercomputers/My laptop/Desktop/ResumeRecommenderMLops/data/constants/KB/detailed_job_descriptions.json"
p2="/content/drive/Othercomputers/My laptop/Desktop/ResumeRecommenderMLops/data/processed/serpapi/2026-01.csv"
p3="/content/drive/Othercomputers/My laptop/Desktop/ResumeRecommenderMLops/data/constants/wiki_llm_training_resume_project.jsonl"

In [ ]:
prompt="""
You are a Knowledge Graph Extraction Engine for a Career Advisory System.

Your task is to extract structured knowledge that explains:
1. WHAT a job role requires.
2. WHY those requirements exist (conceptual grounding).
3. HOW skills transfer across roles (gap and similarity reasoning).

This knowledge graph will be used to explain to candidates:
- Why a job is their best match
- Why other roles are second or third best
- What skills or concepts they should learn next

You must strictly follow the ontology, rules, and output format below.
If something does not fit the rules, DO NOT output it.

--------------------------------------------------
ONTOLOGY CONTRACT (STRICT)
--------------------------------------------------

ALLOWED NODE TYPES:
JobPosting
JobRole
Skill
ProgrammingLanguage
Framework
Tool
Platform
CloudService
Database
Concept
Company
Location

--------------------------------------------------
ALLOWED RELATIONS (SEMANTIC)
--------------------------------------------------

-- Job Structure --
POSTED_BY          (JobPosting -> Company)
LOCATED_IN         (Company -> Location)
IS_FOR_ROLE        (JobPosting -> JobRole)
RELATED_ROLE       (JobRole -> JobRole)

-- Role Requirements (EXPLANATION-ORIENTED) --
REQUIRES_SKILL     (JobRole -> Skill)
REQUIRES_TOOL      (JobRole -> Tool | Framework | Database | Platform | CloudService)
REQUIRES_LANGUAGE  (JobRole -> ProgrammingLanguage)
REQUIRES_CONCEPT   (JobRole -> Concept)

-- Technical Reasoning (WHY / HOW) --
USES_LANGUAGE       (Framework | Tool -> ProgrammingLanguage)
IMPLEMENTS_CONCEPT  (Tool | Framework | ProgrammingLanguage -> Concept)
IS_SIMILAR_TO       (Tool | Framework | Database | CloudService -> same type)
BUILT_WITH          (Tool -> ProgrammingLanguage)
CREATED_BY          (ProgrammingLanguage | Framework | Tool -> Company)

--------------------------------------------------
ALLOWED TRIPLE PATTERNS (MANDATORY)
--------------------------------------------------

You may emit a triplet ONLY if it matches one of the following exact patterns:

-- Job Posting Context --
(JobPosting) IS_FOR_ROLE (JobRole)
(JobPosting) POSTED_BY (Company)
(Company) LOCATED_IN (Location)

-- Role Definition Context --
(JobRole) REQUIRES_SKILL (Skill)
(JobRole) REQUIRES_TOOL (Tool | Framework | Database | Platform | CloudService)
(JobRole) REQUIRES_LANGUAGE (ProgrammingLanguage)
(JobRole) REQUIRES_CONCEPT (Concept)
(JobRole) RELATED_ROLE (JobRole)

-- Technical / Knowledge Context --
(Framework | Tool | ProgrammingLanguage) IMPLEMENTS_CONCEPT (Concept)
(Framework | Tool) USES_LANGUAGE (ProgrammingLanguage)
(Tool | Framework | Database) IS_SIMILAR_TO (same type)
(Tool) BUILT_WITH (ProgrammingLanguage)
(ProgrammingLanguage | Framework | Tool) CREATED_BY (Company)

If a candidate triplet does NOT match these patterns → DISCARD IT.

--------------------------------------------------
FORBIDDEN OUTPUT (NEVER EMIT)
--------------------------------------------------

Do NOT output:
- Responsibilities or actions
- Sentences or long phrases as nodes
- Soft skills (communication, leadership, teamwork)
- Abstract fillers (systems, techniques, solutions)
- Version numbers or editions
- Salary, experience ranges, dates
- Statistics or percentages

--------------------------------------------------
SOURCE-AWARE RULES
--------------------------------------------------

IF INPUT SOURCE IS JOB DESCRIPTION (CSV):
- Treat the record as a JobPosting.
- Always map JobPosting to a normalized JobRole using IS_FOR_ROLE.
- Extract only explicitly mentioned skills, tools, frameworks, languages, or databases.
- Ignore benefits, culture, or generic HR text.

IF INPUT SOURCE IS ROLE DEFINITION (JSON):
- Subject is the JobRole.
- Extract core technical expectations as REQUIRES_* relations.
- Extract adjacent or alternative roles using RELATED_ROLE.

IF INPUT SOURCE IS WIKI ARTICLE:
- DO NOT emit JobPosting or JobRole.
- Focus only on ProgrammingLanguage, Framework, Tool, Concept.
- Emit Company ONLY if it is the creator of the technology (CREATED_BY).
- Emit EXACTLY ONE definition object for the primary entity described.

--------------------------------------------------
WIKI-SPECIFIC: DEFINITION EXTRACTION (MANDATORY)
--------------------------------------------------

If and ONLY if the input source is a Wiki article, return a definition object
focused on UTILITY and EXPLANATION.

Definition schema:
- definition: ONE sentence explaining what it is and why it is useful.
- summary: 2 sentences explaining real-world usage.
- key_characteristics: 3–5 short technical bullet points.
- source: provenance string (e.g. "wikipedia").

Do NOT include history, dates, or trivia.

--------------------------------------------------
ENTITY NORMALIZATION RULES
--------------------------------------------------

- Canonical names only (e.g., "JS" -> "JavaScript").
- Singular form only.
- Title Case for entity names.
- Disambiguate ambiguous terms using context.
- Prefer CloudService over Platform for AWS, Azure, GCP.

--------------------------------------------------
OUTPUT FORMAT (STRICT JSON ONLY)
--------------------------------------------------

Return ONLY a valid JSON object. No markdown. No explanations.

{
  "triplets": [
    {
      "subject": "string",
      "subject_type": "string (from ALLOWED NODE TYPES)",
      "relation": "string (from ALLOWED RELATIONS)",
      "object": "string",
      "object_type": "string (from ALLOWED NODE TYPES)"
    }
  ],
  "definitions": [
    {
      "name": "string",
      "node_type": "string (from ALLOWED NODE TYPES)",
      "definition": "string",
      "summary": "string",
      "key_characteristics": ["string"],
      "source": "string"
    }
  ]
}
"""

In [ ]:
!pip install -q google-genai pandas tqdm groq

In [ ]:
from google.colab import userdata

api_key = userdata.get('temp_gemini')

MAKING TRIPLETS

In [ ]:
import os
import json
import pandas as pd
from google import genai
from google.genai import types
from tqdm.notebook import tqdm
import time

# ==========================================
# 1. CONFIGURATION
# ==========================================
class Config:
    # API Configuration
    API_KEY = api_key  # <--- PASTE YOUR KEY HERE
    MODEL_NAME = "gemini-2.5-flash"

    # File Paths
    PATH_ROLES = "/content/drive/Othercomputers/My laptop/Desktop/ResumeRecommenderMLops/data/constants/KB/detailed_job_descriptions.json"
    PATH_JOBS = "/content/drive/Othercomputers/My laptop/Desktop/ResumeRecommenderMLops/data/processed/serpapi/2026-01.csv"
    PATH_WIKI = "/content/drive/Othercomputers/My laptop/Desktop/ResumeRecommenderMLops/data/constants/wiki_llm_training_resume_project.jsonl"

    # Batching Settings
    BATCH_SIZE_CHARS = 53000  # Character limit per batch (safe for context)

    # Output
    LOCAL_OUTPUT_FILE = "knowledge_graph_extracted_DAY13.jsonl"

    # Backup Settings (Google Drive)
    # Ensure this folder exists in your Drive!
    DRIVE_BACKUP_DIR = "/content/drive/MyDrive/rr_mlops"
    DRIVE_BACKUP_PATH = os.path.join(DRIVE_BACKUP_DIR, LOCAL_OUTPUT_FILE)

    RESUME_FROM_BATCH= 153

# ==========================================
# 2. SYSTEM PROMPT (CONSTANTS)
# ==========================================
SYSTEM_PROMPT = """
You are a Knowledge Graph Extraction Engine for a Career Advisory System.

Your task is to extract structured knowledge that enables EXPLANATION, not just storage.

This knowledge graph must support:
1. WHY a job is the best match for a candidate
2. WHY other roles are second or third best
3. WHAT specific skills or concepts the candidate is missing
4. HOW skills transfer across roles via shared concepts

You must STRICTLY follow the ontology, rules, and output format below.
If something does not fit the rules, DO NOT output it.

--------------------------------------------------
ONTOLOGY CONTRACT (STRICT)
--------------------------------------------------

ALLOWED NODE TYPES:
JobPosting
JobRole
Skill
ProgrammingLanguage
Framework
Tool
Platform
CloudService
Database
Concept
Company
Location

--------------------------------------------------
CORE SEMANTIC PRINCIPLE (MANDATORY)
--------------------------------------------------

The graph MUST separate:
- EXECUTION (Skill)
- EXPLANATION (Concept)

This separation is NON-NEGOTIABLE.

--------------------------------------------------
SKILL vs CONCEPT CLASSIFICATION (CRITICAL RULE)
--------------------------------------------------

Skill = "Can this be directly practiced, implemented, or demonstrated?"

Concept = "Does this explain WHY or WHAT principle something is based on?"

HARD CONSTRAINT:
If a Skill candidate can be rewritten as
"Understanding of X" or "Knowledge of X",
it MUST be classified as Concept.

ANCHORING RULE:
Every Skill emitted for a JobRole SHOULD be supported
by at least one Concept via either:
- JobRole REQUIRES_CONCEPT
- Tool IMPLEMENTS_CONCEPT

If no Concept exists add one according to your knowledge.


VALID Skill examples:
- SQL Query Writing
- REST API Development
- Feature Engineering
- Model Deployment
- Prompt Engineering
- Kubernetes Troubleshooting
- CI/CD Pipeline Implementation
- Data Visualization
- Unit Testing
- API Integration

INVALID as Skill (MUST be Concept):
- Machine Learning
- Deep Learning
- Object-Oriented Programming
- System Design
- Cloud Architecture
- Distributed Systems
- Microservices Architecture
- Natural Language Processing
- Security Best Practices
- Data Modeling
- Statistics

RULES:
- If a term explains WHY → Concept
- If a term explains HOW → Skill
- If uncertain → Concept
- NEVER attach vague domains as Skill

--------------------------------------------------
TOOL vs FRAMEWORK vs PLATFORM (STRICT DEFINITIONS)
--------------------------------------------------
Classify technical entities using these rules:

1. FRAMEWORK: "Inverts control". It calls your code. It provides a scaffold.
   - EXAMPLES: React, Angular, Vue, Django, Spring Boot, TensorFlow, PyTorch, Flutter.
   - RULE: If it dictates the architecture of the app, it is a Framework.

2. TOOL: A library, utility, or software used to perform a task. You call it.
   - EXAMPLES: Pandas (Library -> Tool), NumPy (Library -> Tool), Git (CLI Tool), Docker (Container Tool), Jenkins (Build Tool), Jira (Project Tool).
   - RULE: If you import it to do math/data manipulation, it is a Tool (Library).

3. PLATFORM: An environment where software runs or is hosted.
   - EXAMPLES: iOS, Android, Linux, Windows, GitHub (SaaS Platform), GitLab.

--------------------------------------------------
ALLOWED RELATIONS (SEMANTIC)
--------------------------------------------------

-- Job Structure --
POSTED_BY          (JobPosting -> Company)
LOCATED_IN         (Company -> Location)
IS_FOR_ROLE        (JobPosting -> JobRole)
RELATED_ROLE       (JobRole -> JobRole)

-- Role Requirements (EXPLANATION-FIRST) --
REQUIRES_SKILL     (JobRole -> Skill)
REQUIRES_LANGUAGE  (JobRole -> ProgrammingLanguage)
REQUIRES_TOOL      (JobRole -> Tool | Framework | Database | Platform | CloudService)
REQUIRES_CONCEPT   (JobRole -> Concept)

-- Technical Reasoning (WHY / HOW CHAINS) --
IMPLEMENTS_CONCEPT (Tool | Framework | ProgrammingLanguage -> Concept)
USES_LANGUAGE      (Framework | Tool -> ProgrammingLanguage)
BUILT_WITH         (Tool -> ProgrammingLanguage)
IS_SIMILAR_TO      (Tool | Framework | Database -> same type)
CREATED_BY         (ProgrammingLanguage | Framework | Tool -> Company)

--------------------------------------------------
ALLOWED TRIPLE PATTERNS (MANDATORY)
--------------------------------------------------

You may emit a triplet ONLY if it matches one of these exact patterns:

-- Job Posting Context --
(JobPosting) IS_FOR_ROLE (JobRole)
(JobPosting) POSTED_BY (Company)
(Company) LOCATED_IN (Location)

-- Role Definition Context --
(JobRole) REQUIRES_SKILL (Skill)
(JobRole) REQUIRES_LANGUAGE (ProgrammingLanguage)
(JobRole) REQUIRES_TOOL (Tool | Framework | Database | Platform | CloudService)
(JobRole) REQUIRES_CONCEPT (Concept)
(JobRole) RELATED_ROLE (JobRole)

-- Technical Knowledge Context --
(Framework | Tool | ProgrammingLanguage) IMPLEMENTS_CONCEPT (Concept)
(Framework | Tool) USES_LANGUAGE (ProgrammingLanguage)
(Tool | Framework | Database) IS_SIMILAR_TO (same type)
(Tool) BUILT_WITH (ProgrammingLanguage)
(ProgrammingLanguage | Framework | Tool) CREATED_BY (Company)

If a candidate triplet does NOT match these patterns → DISCARD IT.

--------------------------------------------------
FORBIDDEN OUTPUT (NEVER EMIT)
--------------------------------------------------

Do NOT output:
- Responsibilities or actions
- Soft skills (communication, leadership, teamwork)
- Abstract fillers (systems, solutions, techniques)
- Version numbers or editions
- Salary, experience ranges, dates
- Metrics or percentages
- Multi-sentence nodes

--------------------------------------------------
SOURCE-AWARE RULES
--------------------------------------------------

IF INPUT SOURCE IS JOB DESCRIPTION (CSV):
- Treat each record as a JobPosting.
- ALWAYS map JobPosting → JobRole using IS_FOR_ROLE.
- Extract ONLY explicitly mentioned technical requirements.
- Do NOT infer skills not written in text.

IF INPUT SOURCE IS ROLE DEFINITION (JSON):
- Subject is the JobRole.
- Extract REQUIRED skills, tools, languages, and concepts.
- Extract adjacent or alternative roles using RELATED_ROLE.

IF INPUT SOURCE IS WIKI ARTICLE:
- DO NOT emit JobPosting or JobRole.
- Focus only on ProgrammingLanguage, Framework, Tool, Concept.
- Emit Company ONLY if it is the creator (CREATED_BY).
- Emit EXACTLY ONE definition object for the primary entity.

--------------------------------------------------
WIKI-SPECIFIC: DEFINITION EXTRACTION (MANDATORY)
--------------------------------------------------

If and ONLY if the input source is a Wiki article, return ONE definition object.

Definition MUST focus on UTILITY for career understanding.

Schema:
- definition: ONE sentence explaining what it is and why it is useful.
- summary: 2 sentences explaining where and why it is used.
- key_characteristics: 3–5 short, technical bullet points.
- source: provenance string (e.g. "wikipedia").

Do NOT include history, dates, or trivia.

--------------------------------------------------
ENTITY NORMALIZATION RULES
--------------------------------------------------

- Canonical names only (JS → JavaScript).
- Singular form only.
- Title Case for entities.
- Disambiguate using context.
- Prefer CloudService over Platform for AWS, Azure, GCP.

--------------------------------------------------
OUTPUT FORMAT (STRICT JSON ONLY)
--------------------------------------------------

Return ONLY a valid JSON object. No markdown. No explanations.

{
  "triplets": [
    {
      "subject": "string",
      "subject_type": "string",
      "relation": "string",
      "object": "string",
      "object_type": "string"
    }
  ],
  "definitions": [
    {
      "name": "string",
      "node_type": "string",
      "definition": "string",
      "summary": "string",
      "key_characteristics": ["string"],
      "source": "string"
    }
  ]
}

"""

GROQ ANGLE(GPT-oss-120b)



In [ ]:
"""import requests
import json
import os
import shutil
import time
from tqdm.notebook import tqdm

def main_gpt_oss():
    print("🔧 Setting up OpenAI GPT-OSS-120B pipeline via Groq...")

    # 1. API Setup
    # Use 'openai/gpt-oss-120b' for ultra-high reasoning
    MODEL = "openai/gpt-oss-120b"
    api_key = userdata.get('groq_key')
    url = "https://api.groq.com/openai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # 2. Setup Generators
    role_gen = DataIngestor.read_roles(Config.PATH_ROLES)
    job_gen = DataIngestor.read_jobs_csv(Config.PATH_JOBS)
    wiki_gen = DataIngestor.read_wiki(Config.PATH_WIKI)

    # 3. Create Batcher
    # REDUCED to 40k for 120B model due to strict TPM (Tokens Per Minute) limits
    BATCH_SIZE = 40000
    all_batches = batch_generator([role_gen, job_gen, wiki_gen], BATCH_SIZE)

    print(f"\n🚀 Starting High-Reasoning Extraction (Resuming from Batch {Config.RESUME_FROM_BATCH})...\n")

    batch_idx = 0
    for batch_text in tqdm(all_batches, desc="120B Extraction"):
        batch_idx += 1

        if batch_idx < Config.RESUME_FROM_BATCH:
            if batch_idx % 10 == 0: print(f"⏩ Skipping Batch {batch_idx}")
            continue

        payload = {
            "model": MODEL,
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"EXTRACT KNOWLEDGE FROM THIS BATCH:\n{batch_text}"}
            ],
            "response_format": {"type": "json_object"},
            "temperature": 0.1,
            "reasoning_effort": "high" # Specific to GPT-OSS models for better logic
        }

        # --- RETRY LOGIC FOR THE 120B MODEL ---
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.post(url, headers=headers, json=payload)

                if response.status_code == 429:
                    wait_time = 60 * (attempt + 1)
                    print(f"\n⏳ Rate limit hit. Cooling down for {wait_time}s (Attempt {attempt+1}/{max_retries})...")
                    time.sleep(wait_time)
                    continue

                response.raise_for_status()

                # Parse and Save
                res_json = response.json()
                raw_content = res_json['choices'][0]['message']['content']

                if raw_content:
                    json_data = json.loads(raw_content)

                    with open(Config.LOCAL_OUTPUT_FILE, 'a', encoding='utf-8') as f_out:
                        json.dump(json_data, f_out)
                        f_out.write("\n")

                    if os.path.exists(Config.DRIVE_BACKUP_DIR):
                        shutil.copy2(Config.LOCAL_OUTPUT_FILE, Config.DRIVE_BACKUP_PATH)

                    triplets = len(json_data.get('triplets', []))
                    print(f"✅ Batch {batch_idx}: Saved {triplets} triplets (GPT-OSS 120B)")

                # SUCCESS: Wait for TPM to reset before next batch
                # 80s is the 'Safe Zone' for the 120B model on free tiers
                time.sleep(80)
                break

            except Exception as e:
                print(f"💀 Batch {batch_idx}: Error - {str(e)}")
                if attempt == max_retries - 1:
                    print("❌ Max retries reached. Stopping to save progress.")
                    return
                time.sleep(10)
    print(f"\n🎉 High-Reasoning session completed!")"""

In [ ]:
class DataIngestor:
    """Reads raw files and yields formatted text strings."""

    @staticmethod
    def read_roles(path):
        """Reads the Detailed Job Descriptions JSON."""
        print(f"Loading Roles from: {path}")
        try:
            with open(path, 'r') as f:
                data = json.load(f)
                if not isinstance(data, list): data = [data]

                for item in data:
                    role_name = item.get("job_title", item.get("role", "Unknown Role"))
                    yield (
                        f"SOURCE: ROLE_DEFINITION_JSON\n"
                        f"ROLE_NAME: {role_name}\n"
                        f"CONTENT: {json.dumps(item)}\n\n"
                    )
        except Exception as e:
            print(f"Error reading roles: {e}")

    @staticmethod
    def read_jobs_csv(path):
        """Reads the SerpAPI Job CSV."""
        print(f"Loading Jobs from: {path}")
        try:
            df = pd.read_csv(path)
            cols = ['title', 'company_name', 'location', 'description']
            for c in cols:
                if c not in df.columns: df[c] = "Unknown"

            for _, row in df[cols].iterrows():
                # Jobs are already concise, keeping the 2500 char cap
                desc = str(row['description'])[:5000]
                text = (
                    f"SOURCE: JOB_DESCRIPTION_CSV\n"
                    f"Job Title: {row['title']}\n"
                    f"Company: {row['company_name']}\n"
                    f"Description: {desc}...\n\n"
                )
                yield text
        except Exception as e:
            print(f"Error reading jobs: {e}")

    @staticmethod
    def read_wiki(path):
        """Reads the Wiki JSONL file with a 1500 word cap per article."""
        print(f"Loading Wiki from: {path}")
        try:
            with open(path, 'r') as f:
                for line in f:
                    try:
                        item = json.loads(line)
                        title = item.get('title', 'Unknown')
                        content = item.get('text', '')

                        # NEW: Word-based capping (Approx 6k-8k characters)
                        # We split by space, take 1500 words, and join back.
                        words = content.split()
                        capped_content = " ".join(words[:1500])

                        yield (
                            f"SOURCE: WIKI_ARTICLE\n"
                            f"PRIMARY_ENTITY: {title}\n"
                            f"CONTENT: {capped_content}\n\n"
                        )
                    except json.JSONDecodeError: continue
        except Exception as e:
            print(f"Error reading Wiki: {e}")

In [ ]:
def batch_generator(iterables, max_chars=53500):
    """
    Combines generators into large batches (80k chars).
    Ensures batches DO NOT mix SOURCE types.
    """
    for source_gen in iterables:
        current_batch = []
        current_len = 0

        for text_chunk in source_gen:
            chunk_len = len(text_chunk)

            # Safety for individual chunks larger than the total batch limit
            if chunk_len > max_chars:
                if current_batch:
                    yield "".join(current_batch)
                    current_batch = []
                    current_len = 0
                    yield text_chunk[:max_chars]
                    continue

            if current_len + chunk_len > max_chars and current_batch:
                yield "".join(current_batch)
                current_batch = []
                current_len = 0

            current_batch.append(text_chunk)
            current_len += chunk_len

        if current_batch:
            yield "".join(current_batch)

In [ ]:
import os
import shutil
import json
import time
from tqdm.notebook import tqdm
from google import genai
from google.genai import types

def main():
    print("🔧 Setting up pipeline...")
    client = genai.Client(api_key=Config.API_KEY)

    # Setup Generators
    role_gen = DataIngestor.read_roles(Config.PATH_ROLES)
    job_gen = DataIngestor.read_jobs_csv(Config.PATH_JOBS)
    wiki_gen = DataIngestor.read_wiki(Config.PATH_WIKI)

    # Initialize Batcher with your 120k Config
    all_batches = batch_generator([role_gen, job_gen, wiki_gen], Config.BATCH_SIZE_CHARS)

    print(f"\n🚀 Extraction Pipeline (Resume: {Config.RESUME_FROM_BATCH} | Size: {Config.BATCH_SIZE_CHARS})...\n")

    batch_idx = 0
    for batch_text in tqdm(all_batches, desc="Batch Progress"):
        batch_idx += 1

        if batch_idx < Config.RESUME_FROM_BATCH:
            if batch_idx % 5 == 0: print(f"⏩ Skipping Batch {batch_idx}")
            continue

        try:
            full_content = f"{SYSTEM_PROMPT}\n\n=== DATA TO EXTRACT ===\n{batch_text}"

            response = client.models.generate_content(
                model=Config.MODEL_NAME,
                contents=full_content,
                config=types.GenerateContentConfig(response_mime_type="application/json")
            )

            if response.text:
                try:
                    json_data = json.loads(response.text)

                    # 1. Atomic Local Save
                    with open(Config.LOCAL_OUTPUT_FILE, 'a', encoding='utf-8') as f_out:
                        json.dump(json_data, f_out)
                        f_out.write("\n")

                    # 2. Frequent Drive Sync (Every 2 batches to save overhead)
                    if batch_idx % 2 == 0 and os.path.exists(Config.DRIVE_BACKUP_DIR):
                        shutil.copy2(Config.LOCAL_OUTPUT_FILE, Config.DRIVE_BACKUP_PATH)

                    triplet_count = len(json_data.get('triplets', []))
                    print(f"✅ Batch {batch_idx}: {triplet_count} triplets saved.")

                except json.JSONDecodeError:
                    print(f"❌ Batch {batch_idx}: Invalid JSON response.")

            # FIXED: Fixed 15s sleep to respect 5 RPM and 250k TPM limits
            time.sleep(20)

        except Exception as e:
            if "429" in str(e) or "RESOURCE_EXHAUSTED" in str(e):
                print(f"\n🛑 DAILY QUOTA (20 RPD) REACHED at Batch {batch_idx}. Resume tomorrow.")
                # Final emergency sync before stopping
                shutil.copy2(Config.LOCAL_OUTPUT_FILE, Config.DRIVE_BACKUP_PATH)
                break
            else:
                print(f"💀 Batch {batch_idx} Error: {str(e)}")
                time.sleep(30) # Extended cooldown on general errors

    print(f"\n🎉 Progress saved. Last processed: Batch {batch_idx}")

In [ ]:
if __name__== "__main__":
  main()

PROCESSING TRIPLETS (VALIDATION LAYER)

In [ ]:
import json
import re

# ==========================================
# 1. CONFIGURATION
# ==========================================
INPUT_FILE = "knowledge_graph_extracted_DAY13.jsonl"
OUTPUT_FILE = "knowledge_graph_clean_DAY13.jsonl"

# ==========================================
# 2. ONTOLOGY ENFORCEMENT (THE "TRUTH")
# ==========================================

# A. Type Overrides (Fixing LLM Misclassifications)
TYPE_OVERRIDES = {
    # Data Science / ML
    "Pandas": "Tool", "NumPy": "Tool", "Scikit-Learn": "Framework",
    "Keras": "Framework", "PyTorch": "Framework", "TensorFlow": "Framework",
    "OpenCV": "Tool", "NLTK": "Tool", "SpaCy": "Tool",
    "Matplotlib": "Tool", "Seaborn": "Tool", "D3.js": "Tool", "MLflow": "Tool",
    # DevOps / Cloud
    "Docker": "Tool", "Kubernetes": "Tool", "Terraform": "Tool",
    "Jenkins": "Tool", "Git": "Tool", "Ansible": "Tool",
    "AWS": "CloudService", "Azure": "CloudService", "GCP": "CloudService",
    # Concepts
    "Statistics": "Concept", "Probability": "Concept", "Linear Algebra": "Concept",
    "Calculus": "Concept", "Machine Learning": "Concept", "Deep Learning": "Concept",
    "Data Science": "Concept", "Computer Vision": "Concept", "NLP": "Concept",
    "CI/CD": "Concept", "DevOps": "Concept", "MLOps": "Concept",
    "Agile": "Concept", "Scrum": "Concept", "REST": "Concept",
    "Microservices": "Concept", "System Design": "Concept", "Algorithms": "Concept",
    "Data Structures": "Concept", "Object-Oriented Programming": "Concept"
}

# B. Canonical Name Mapping
CANONICAL_NAMES = {
    "react.js": "React", "reactjs": "React",
    "vue.js": "Vue", "vuejs": "Vue", "vue": "Vue",
    "node.js": "Node.js", "nodejs": "Node.js", "node": "Node.js",
    "express.js": "Express", "expressjs": "Express",
    "next.js": "Next.js", "nextjs": "Next.js",
    "amazon web services": "AWS", "aws": "AWS",
    "google cloud platform": "GCP", "google cloud": "GCP", "gcp": "GCP",
    "microsoft azure": "Azure", "azure": "Azure",
    "postgresql": "PostgreSQL", "postgres": "PostgreSQL",
    "mongodb": "MongoDB", "mongo": "MongoDB",
    "mssql": "SQL Server", "sql server": "SQL Server",
    "tensorflow": "TensorFlow", "tf": "TensorFlow",
    "pytorch": "PyTorch",
    "scikit-learn": "Scikit-Learn", "sklearn": "Scikit-Learn",
    "mlflow": "MLflow",
    "golang": "Go", "go lang": "Go",
    "c#": "C#", "c sharp": "C#", "c++": "C++",
    "dotnet": ".NET", ".net": ".NET", ".net core": ".NET Core"
}

# C. Patterns to Strip
BAD_PATTERNS = [
    r"\bability to\b", r"\bexperience in\b", r"\bknowledge of\b",
    r"\bresponsible for\b", r"\bworking with\b", r"\bunderstanding of\b",
    r"\bfamiliarity with\b", r"\bproven track record\b", r"\bproficient in\b"
]

ADJECTIVES = [
    r"^Strong\s+", r"^Advanced\s+", r"^Basic\s+", r"^Senior\s+", r"^Junior\s+",
    r"^Hands-on\s+", r"^Expertise in\s+", r"\s+Skills$", r"\s+Development$"
]

# D. Injected Logic
INJECTED_EDGES = {
    "PyTorch": [("IMPLEMENTS_CONCEPT", "Deep Learning", "Concept")],
    "TensorFlow": [("IMPLEMENTS_CONCEPT", "Deep Learning", "Concept")],
    "Scikit-Learn": [("IMPLEMENTS_CONCEPT", "Machine Learning", "Concept")],
    "React": [("IMPLEMENTS_CONCEPT", "Frontend Development", "Concept"), ("IMPLEMENTS_CONCEPT", "Component-Based Architecture", "Concept")],
    "Docker": [("IMPLEMENTS_CONCEPT", "Containerization", "Concept")],
    "Kubernetes": [("IMPLEMENTS_CONCEPT", "Orchestration", "Concept")],
    "PostgreSQL": [("IMPLEMENTS_CONCEPT", "Relational Database", "Concept")],
    "MongoDB": [("IMPLEMENTS_CONCEPT", "NoSQL", "Concept")],
    "REST": [("IS_SUBTOPIC_OF", "Architectural Style", "Concept")],
    "CI/CD": [("IS_SUBTOPIC_OF", "DevOps Practice", "Concept")]
}

ALLOWED_PATTERNS = {
    ("JobRole", "REQUIRES_SKILL", "Skill"),
    ("JobRole", "REQUIRES_KNOWLEDGE", "Concept"),
    ("JobRole", "REQUIRES_CONCEPT", "Concept"),
    ("JobRole", "REQUIRES_TOOL", "Tool"),
    ("JobRole", "REQUIRES_TOOL", "Framework"),
    ("JobRole", "REQUIRES_TOOL", "Database"),
    ("JobRole", "REQUIRES_TOOL", "Platform"),
    ("JobRole", "REQUIRES_TOOL", "CloudService"),
    ("JobRole", "REQUIRES_LANGUAGE", "ProgrammingLanguage"),
    ("Tool", "IMPLEMENTS_CONCEPT", "Concept"),
    ("Framework", "IMPLEMENTS_CONCEPT", "Concept"),
    ("ProgrammingLanguage", "IMPLEMENTS_CONCEPT", "Concept"),
    ("Tool", "USES_LANGUAGE", "ProgrammingLanguage"),
    ("Framework", "USES_LANGUAGE", "ProgrammingLanguage"),
    ("Concept", "IS_SUBTOPIC_OF", "Concept")
}

# ==========================================
# 3. CLEANING LOGIC
# ==========================================
def clean_name(name):
    if not name: return None
    name = name.strip()

    # 1. Strip Adjectives
    for pattern in ADJECTIVES:
        name = re.sub(pattern, "", name, flags=re.IGNORECASE)

    # 2. Check for Vague Phrases
    for pat in BAD_PATTERNS:
        if re.search(pat, name.lower()):
            return None

    # 3. Canonicalize
    lower_name = name.lower()
    if lower_name in CANONICAL_NAMES:
        return CANONICAL_NAMES[lower_name]

    # 4. Default Title Case
    if lower_name in ["sql", "api", "etl", "elt", "bi"]:
        return name.upper()

    return name.title()

def validate_triplet(t):
    # 1. Clean Names (Defensive check for missing keys)
    s_raw = t.get('subject')
    o_raw = t.get('object')

    if not s_raw or not o_raw: return None

    s_clean = clean_name(s_raw)
    o_clean = clean_name(o_raw)

    if not s_clean or not o_clean: return None

    # 2. Apply Type Overrides
    # FIX: Use .get() with a default value to prevent KeyError
    s_type_raw = t.get('subject_type', 'Concept')
    o_type_raw = t.get('object_type', 'Concept')

    s_type = TYPE_OVERRIDES.get(s_clean, s_type_raw)
    o_type = TYPE_OVERRIDES.get(o_clean, o_type_raw)

    # 3. Subject-Side Coercion
    if s_clean in TYPE_OVERRIDES and TYPE_OVERRIDES[s_clean] == "Concept":
        s_type = "Concept"

    # 4. Strict Logic: Concepts cannot be Skills
    relation = t.get('relation', 'RELATED_TO') # Default relation if missing
    if o_clean in TYPE_OVERRIDES and TYPE_OVERRIDES[o_clean] == "Concept":
        if relation == "REQUIRES_SKILL":
            relation = "REQUIRES_CONCEPT"

    return {
        "subject": s_clean, "subject_type": s_type,
        "relation": relation,
        "object": o_clean, "object_type": o_type
    }

# ==========================================
# 4. MAIN PROCESSING
# ==========================================
def main():
    print("🧹 Starting V3 Canonical Validation (Defensive Mode)...")

    seen_hashes = set()
    valid_count = 0

    try:
        with open(INPUT_FILE, 'r') as f_in, open(OUTPUT_FILE, 'w') as f_out:

            for line in f_in:
                try:
                    data = json.loads(line)
                    triplets = data.get("triplets", [])

                    if not isinstance(triplets, list): continue # Guard against malformed JSON

                    for raw in triplets:
                        clean_t = validate_triplet(raw)
                        if not clean_t: continue

                        # --- WRITE ORIGINAL ---
                        h = f"{clean_t['subject']}|{clean_t['relation']}|{clean_t['object']}"
                        if h not in seen_hashes:
                            f_out.write(json.dumps(clean_t) + "\n")
                            seen_hashes.add(h)
                            valid_count += 1

                        # --- HYDRATION (INJECT MISSING EDGES) ---
                        if clean_t['subject'] in INJECTED_EDGES:
                            for rel, target, target_type in INJECTED_EDGES[clean_t['subject']]:
                                if (clean_t['subject_type'], rel, target_type) not in ALLOWED_PATTERNS:
                                    if rel not in ["IMPLEMENTS_CONCEPT", "IS_SUBTOPIC_OF"]:
                                        continue

                                new_t = {
                                    "subject": clean_t['subject'], "subject_type": clean_t['subject_type'],
                                    "relation": rel,
                                    "object": target, "object_type": target_type
                                }
                                h_new = f"{new_t['subject']}|{new_t['relation']}|{new_t['object']}"
                                if h_new not in seen_hashes:
                                    f_out.write(json.dumps(new_t) + "\n")
                                    seen_hashes.add(h_new)
                                    valid_count += 1

                        if clean_t['object'] in INJECTED_EDGES:
                            for rel, target, target_type in INJECTED_EDGES[clean_t['object']]:
                                new_t = {
                                    "subject": clean_t['object'], "subject_type": clean_t['object_type'],
                                    "relation": rel,
                                    "object": target, "object_type": target_type
                                }
                                h_new = f"{new_t['subject']}|{new_t['relation']}|{new_t['object']}"
                                if h_new not in seen_hashes:
                                    f_out.write(json.dumps(new_t) + "\n")
                                    seen_hashes.add(h_new)
                                    valid_count += 1

                except json.JSONDecodeError: pass

        print(f"✅ Success! Written {valid_count} canonicalized triplets to {OUTPUT_FILE}")

    except FileNotFoundError:
        print(f"❌ Input file '{INPUT_FILE}' not found.")

if __name__ == "__main__":
    main()

GENERATING CYPHERS


In [ ]:
import json

# ==========================================
# 1. CONFIGURATION
# ==========================================
INPUT_FILE = "knowledge_graph_clean_DAY13.jsonl"
OUTPUT_FILE = "ingest_graph_DAY13.cypher"
BATCH_SIZE = 1000  # Safe batch size for AuraDB

# ==========================================
# 2. HELPER: CHUNKING
# ==========================================
def chunker(seq, size):
    """Yields chunks of a specific size from a list."""
    for i in range(0, len(seq), size):
        yield seq[i:i + size]

# ==========================================
# 3. GENERATORS
# ==========================================
def generate_constraints(types):
    """Generates unique constraints for every node type found."""
    return [
        f"CREATE CONSTRAINT {t.lower()}_uniq IF NOT EXISTS FOR (n:{t}) REQUIRE n.name IS UNIQUE;"
        for t in sorted(list(types))
    ]

def generate_node_cypher(triplets):
    """Generates batched UNWIND statements for Node creation."""
    nodes = {}

    # 1. Aggregate unique names per type
    for t in triplets:
        for side in ['subject', 'object']:
            typ = t[f'{side}_type']
            name = t[f'{side}']
            if typ not in nodes: nodes[typ] = set()
            nodes[typ].add(name)

    statements = []
    statements.append("// --- 2. NODE CREATION ---")

    # 2. Generate Batched Queries
    for n_type, names_set in nodes.items():
        all_names = list(names_set)

        for batch in chunker(all_names, BATCH_SIZE):
            json_batch = json.dumps(batch)  # Handles escaping safely

            query = (
                f"// Batch: Create {n_type} nodes\n"
                f"UNWIND {json_batch} AS name "
                f"MERGE (n:{n_type} {{name: name}});"
            )
            statements.append(query)

    return statements

def generate_rel_cypher(triplets):
    """Generates batched UNWIND statements for Relationship creation."""
    rels = {}

    # 1. Aggregate relations by signature
    for t in triplets:
        key = (t['subject_type'], t['relation'], t['object_type'])
        if key not in rels: rels[key] = []

        rels[key].append({"s": t['subject'], "o": t['object']})

    statements = []
    statements.append("// --- 3. RELATIONSHIP CREATION ---")

    # 2. Generate Batched Queries
    for (st, rel, ot), pairs in rels.items():
        for batch in chunker(pairs, BATCH_SIZE):
            json_batch = json.dumps(batch)

            query = (
                f"// Batch: {st} -> {rel} -> {ot}\n"
                f"UNWIND {json_batch} AS row "
                f"MATCH (s:{st} {{name: row.s}}) "
                f"MATCH (o:{ot} {{name: row.o}}) "
                f"MERGE (s)-[:{rel}]->(o);"
            )
            statements.append(query)

    return statements

# ==========================================
# 4. MAIN
# ==========================================
def main():
    print("⚙️ Generating Optimized Cypher Script...")
    triplets = []

    try:
        # Load JSONL
        with open(INPUT_FILE, 'r') as f:
            for line in f:
                if line.strip():
                    try:
                        triplets.append(json.loads(line))
                    except json.JSONDecodeError:
                        continue

        print(f"📊 Loaded {len(triplets)} triplets.")

        # 1. Generate Components
        all_types = set()
        for t in triplets:
            all_types.add(t['subject_type'])
            all_types.add(t['object_type'])

        constraints = generate_constraints(all_types)
        node_queries = generate_node_cypher(triplets)
        rel_queries = generate_rel_cypher(triplets)

        # 2. Write to File with Instructions
        with open(OUTPUT_FILE, 'w') as f:
            # Header Instructions
            f.write("// ======================================================\n")
            f.write("// INSTRUCTIONS FOR NEO4J BROWSER:\n")
            f.write("// 1. Ensure 'Connect result nodes' is unchecked in settings (optional, for speed).\n")
            f.write("// 2. Ensure 'Enable multi-statement query editor' is CHECKED.\n")
            f.write("// 3. Copy-paste this entire file and run it.\n")
            f.write("// ======================================================\n\n")

            f.write("// --- 1. CONSTRAINTS ---\n")
            f.write("\n".join(constraints) + "\n\n")

            # Double newline (\n\n) ensures clear separation for the Browser parser
            f.write("\n\n".join(node_queries) + "\n\n")
            f.write("\n\n".join(rel_queries) + "\n")

        print(f"✅ Success! Script saved to: {OUTPUT_FILE}")
        print(f"   - Node Batches: {len(node_queries)}")
        print(f"   - Relationship Batches: {len(rel_queries)}")

    except FileNotFoundError:
        print("❌ Error: Input file not found. Run validation script first.")

if __name__ == "__main__":
    main()

PUSHING CYPHERS TO GRAPH DB

In [ ]:
import json
import os
import glob

# ==========================================
# CONFIGURATION
# ==========================================
SOURCE_DIR = "/content/drive/MyDrive/rr_mlops"  # Where your .jsonl files are
OUTPUT_FILE = os.path.join(SOURCE_DIR, "ingest_fixed.cypher")
BATCH_SIZE = 1000  # Smaller batch size = Safer ingestion

def generate_clean_cypher():
    print(f"🧹 Scanning {SOURCE_DIR} for JSONL files...")
    files = glob.glob(os.path.join(SOURCE_DIR, "*.jsonl"))

    if not files:
        print("❌ No .jsonl files found! Cannot regenerate.")
        return

    # 1. Collect Valid Triplets
    # We use a set to deduplicate inherently
    unique_triplets = set()

    for f_path in files:
        print(f"   📖 Reading {os.path.basename(f_path)}...")
        with open(f_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    # Extract triplets if valid
                    if "triplets" in data:
                        for t in data["triplets"]:
                            # Create a tuple signature to deduplicate: (sub, sub_type, rel, obj, obj_type)
                            sig = (
                                t.get("subject"),
                                t.get("subject_type"),
                                t.get("relation"),
                                t.get("object"),
                                t.get("object_type")
                            )
                            if all(sig): # Ensure no None values
                                unique_triplets.add(sig)
                except:
                    continue

    print(f"✅ Loaded {len(unique_triplets)} unique triplets.")

    # 2. Structure Data for Batching
    # We separate by Relation Type to optimize Neo4j locking
    # Structure: { "REQUIRES_SKILL": [ {s:..., o:...}, ... ], ... }
    batched_rels = {}
    nodes = set() # (name, type)

    for sub, sub_type, rel, obj, obj_type in unique_triplets:
        # Collect Nodes
        nodes.add((sub, sub_type))
        nodes.add((obj, obj_type))

        # Collect Edge Data
        if rel not in batched_rels:
            batched_rels[rel] = []

        # We store just the names and types needed for the match
        batched_rels[rel].append({
            "s": sub, "st": sub_type,
            "o": obj, "ot": obj_type
        })

    # 3. Write the Clean Cypher Script
    print(f"✍️ Writing to {OUTPUT_FILE}...")
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        f.write("// AUTO-GENERATED VALIDATED CYPHER\n\n")

        # --- PART A: NODES (Batched) ---
        # Group nodes by label for efficient UNWIND
        nodes_by_label = {}
        for name, label in nodes:
            if label not in nodes_by_label: nodes_by_label[label] = []
            nodes_by_label[label].append(name)

        for label, names in nodes_by_label.items():
            # Chunk into batches
            for i in range(0, len(names), BATCH_SIZE):
                batch = names[i:i+BATCH_SIZE]
                # We use json.dumps to ensure quotes/escaping are perfect
                json_batch = json.dumps(batch)

                query = (
                    f"// Create {label} Nodes (Batch {i})\n"
                    f"UNWIND {json_batch} AS name\n"
                    f"MERGE (n:`{label}` {{name: name}});\n\n"
                )
                f.write(query)

        # --- PART B: RELATIONSHIPS (Batched) ---
        for rel_type, edges in batched_rels.items():
            for i in range(0, len(edges), BATCH_SIZE):
                batch = edges[i:i+BATCH_SIZE]
                json_batch = json.dumps(batch)

                # Dynamic Cypher generation based on the first item's types
                # (Assuming uniform types per relation batch for simplicity,
                # but we match on s/st to be safe)

                query = (
                    f"// Create {rel_type} Edges (Batch {i})\n"
                    f"UNWIND {json_batch} AS row\n"
                    f"MATCH (s {{name: row.s}}) WHERE labels(s)[0] = row.st\n"
                    f"MATCH (o {{name: row.o}}) WHERE labels(o)[0] = row.ot\n"
                    f"MERGE (s)-[:`{rel_type}`]->(o);\n\n"
                )
                f.write(query)

    print("🎉 Success! New script is ready for ingestion.")

# Run it
generate_clean_cypher()

In [ ]:
!pip install neo4j~=5.28

In [ ]:
from google.colab import userdata

NEO4J_PASSWORD= userdata.get('neo4j_rr_DB')

In [ ]:
import os
import glob
import json
import time
from neo4j import GraphDatabase, exceptions

# ==========================================
# 1. CONFIGURATION
# ==========================================
class Config:
    SOURCE_DIR = "/content/drive/MyDrive/rr_mlops"

    # DB Credentials
    NEO4J_URI = "neo4j+s://0c60999b.databases.neo4j.io"
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD= NEO4J_PASSWORD

    BATCH_SIZE = 1000
    MAX_RETRIES = 3

    # All node types that need unique constraints
    ONTOLOGY_LABELS = [
        "JobPosting", "JobRole", "Skill", "Concept",
        "ProgrammingLanguage", "Framework", "Tool",
        "Platform", "CloudService", "Database",
        "Company", "Location"
    ]

# ==========================================
# 2. ROBUST UTILS
# ==========================================
def run_with_retry(session, query, parameters, retries=Config.MAX_RETRIES):
    """Executes a query with transient error handling."""
    for attempt in range(retries):
        try:
            result = session.run(query, parameters)
            return result.consume()
        except (exceptions.ServiceUnavailable, exceptions.TransientError) as e:
            if attempt < retries - 1:
                sleep_time = 2 * (attempt + 1)
                print(f"      ⚠️ Transient Error (Attempt {attempt+1}/{retries}). Retrying in {sleep_time}s...")
                time.sleep(sleep_time)
            else:
                raise e

# ==========================================
# 3. SCHEMA SAFETY (Constraints)
# ==========================================
def ensure_constraints(driver):
    print("\n🛡️ Verifying Constraints (Speed & Integrity)...")
    with driver.session() as session:
        for label in Config.ONTOLOGY_LABELS:
            # We use the idempotent syntax (IF NOT EXISTS)
            query = f"CREATE CONSTRAINT unique_{label.lower()} IF NOT EXISTS FOR (n:`{label}`) REQUIRE n.name IS UNIQUE"
            try:
                session.run(query)
                print(f"   ✅ Constraint active: :{label}")
            except Exception as e:
                print(f"   ❌ Failed to set constraint for :{label} -> {e}")

# ==========================================
# 4. DATA PREPARATION (Signature Grouping)
# ==========================================
def load_and_group_data(source_dir):
    print(f"\n📖 Scanning {source_dir} for cleaned data...")
    # SORTED for determinism
    files = sorted(glob.glob(os.path.join(source_dir, "*.jsonl")))

    # 1. Group Nodes by Label: { "JobRole": {"DevOps", "SRE"}, ... }
    nodes_by_label = {}

    # 2. Group Edges by Signature: { ("JobRole", "REQUIRES_SKILL", "Skill"): [{"s": "DevOps", "o": "Docker"}, ...], ... }
    edges_by_signature = {}

    triplet_count = 0

    for f_path in files:
        # Skip raw extraction logs if you have clean/final files
        if "failed" in f_path: continue

        with open(f_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    if "triplets" in data:
                        for t in data["triplets"]:
                            s, st = t.get("subject"), t.get("subject_type")
                            r = t.get("relation")
                            o, ot = t.get("object"), t.get("object_type")

                            if all([s, st, r, o, ot]):
                                # Add to Nodes
                                nodes_by_label.setdefault(st, set()).add(s)
                                nodes_by_label.setdefault(ot, set()).add(o)

                                # Add to Edges (Signature Based)
                                sig = (st, r, ot)
                                edges_by_signature.setdefault(sig, []).append({"s": s, "o": o})
                                triplet_count += 1
                except: continue

    print(f"✅ Parsed {triplet_count} triplets.")
    print(f"   - {len(nodes_by_label)} Node Labels found")
    print(f"   - {len(edges_by_signature)} Unique Relationship Patterns found")

    return nodes_by_label, edges_by_signature

# ==========================================
# 5. INGESTION ENGINE
# ==========================================
def ingest_data(driver, nodes_map, edges_map):
    with driver.session() as session:

        # --- PHASE 1: NODES (The Foundation) ---
        print("\n🏗️ Phase 1: Ingesting Nodes...")
        for label, names in nodes_map.items():
            name_list = list(names)
            total = len(name_list)
            print(f"   🔹 :{label} ({total} items)")

            for i in range(0, total, Config.BATCH_SIZE):
                batch = name_list[i:i+Config.BATCH_SIZE]

                # QUERY: Hardcoded Label -> Fast & Safe
                query = f"UNWIND $batch AS name MERGE (n:`{label}` {{name: name}})"

                try:
                    run_with_retry(session, query, {"batch": batch})
                except Exception as e:
                    print(f"      ❌ Failed batch {i}: {e}")

        # --- PHASE 2: RELATIONSHIPS (The Context) ---
        print("\n🔗 Phase 2: Ingesting Relationships...")
        for (s_label, rel_type, o_label), edge_list in edges_map.items():
            total = len(edge_list)
            print(f"   🔸 ({s_label}) -[:{rel_type}]-> ({o_label}) : {total} items")

            for i in range(0, total, Config.BATCH_SIZE):
                batch = edge_list[i:i+Config.BATCH_SIZE]

                # QUERY: Dynamic Construction, Static Execution
                # This fixes the "labels(s)[0]" bug by knowing the label in advance
                query = (
                    f"UNWIND $batch AS row "
                    f"MATCH (s:`{s_label}` {{name: row.s}}) "
                    f"MATCH (o:`{o_label}` {{name: row.o}}) "
                    f"MERGE (s)-[:`{rel_type}`]->(o)"
                )

                try:
                    run_with_retry(session, query, {"batch": batch})
                except Exception as e:
                    print(f"      ❌ Failed batch {i}: {e}")

    print("\n🎉 Ingestion Pipeline Complete!")

# ==========================================
# 6. MAIN
# ==========================================
def main():
    driver = None
    try:
        driver = GraphDatabase.driver(
            Config.NEO4J_URI,
            auth=(Config.NEO4J_USER, Config.NEO4J_PASSWORD)
        )
        driver.verify_connectivity()

        # 1. Safety
        ensure_constraints(driver)

        # 2. Preparation
        nodes, edges = load_and_group_data(Config.SOURCE_DIR)

        # 3. Execution
        ingest_data(driver, nodes, edges)

    except Exception as e:
        print(f"💀 Critical Error: {e}")
    finally:
        if driver: driver.close()

if __name__ == "__main__":
    main()


🛡️ Verifying Constraints (Speed & Integrity)...
   ✅ Constraint active: :JobPosting
   ✅ Constraint active: :JobRole
   ✅ Constraint active: :Skill
   ✅ Constraint active: :Concept
   ✅ Constraint active: :ProgrammingLanguage
   ✅ Constraint active: :Framework
   ✅ Constraint active: :Tool
   ✅ Constraint active: :Platform
   ✅ Constraint active: :CloudService
   ✅ Constraint active: :Database
   ✅ Constraint active: :Company
   ✅ Constraint active: :Location

📖 Scanning /content/drive/MyDrive/rr_mlops for cleaned data...
✅ Parsed 35015 triplets.
   - 13 Node Labels found
   - 140 Unique Relationship Patterns found

🏗️ Phase 1: Ingesting Nodes...
   🔹 :JobRole (463 items)
   🔹 :ProgrammingLanguage (239 items)
   🔹 :Framework (578 items)
   🔹 :Tool (1628 items)
   🔹 :Database (147 items)
   🔹 :CloudService (293 items)
   🔹 :Skill (4361 items)
   🔹 :Concept (5219 items)
   🔹 :Platform (329 items)
   🔹 :Company (668 items)
   🔹 :null (1 items)
   🔹 :Location (164 items)
   🔹 :JobPosting (585 items)

🔗 Phase 2: Ingesting Relationships...
   🔸 (JobRole) -[:REQUIRES_LANGUAGE]-> (ProgrammingLanguage) : 1309 items
   🔸 (JobRole) -[:REQUIRES_TOOL]-> (Framework) : 921 items
   🔸 (JobRole) -[:REQUIRES_TOOL]-> (Tool) : 2192 items
   🔸 (JobRole) -[:REQUIRES_TOOL]-> (Database) : 524 items
   🔸 (JobRole) -[:REQUIRES_TOOL]-> (CloudService) : 677 items
   🔸 (JobRole) -[:REQUIRES_SKILL]-> (Skill) : 7280 items
   🔸 (JobRole) -[:REQUIRES_CONCEPT]-> (Concept) : 8733 items
   🔸 (JobRole) -[:RELATED_ROLE]-> (JobRole) : 347 items
   🔸 (ProgrammingLanguage) -[:IMPLEMENTS_CONCEPT]-> (Concept) : 1143 items
   🔸 (Framework) -[:IMPLEMENTS_CONCEPT]-> (Concept) : 1573 items
   🔸 (JobRole) -[:REQUIRES_FRAMEWORK]-> (Framework) : 321 items
   🔸 (JobRole) -[:REQUIRES_DATABASE]-> (Database) : 152 items
   🔸 (JobRole) -[:REQUIRES_PLATFORM]-> (Platform) : 129 items
   🔸 (JobRole) -[:REQUIRES_CLOUDSERVICE]-> (CloudService) : 106 items
   🔸 (Framework) -[:USES_LANGUAGE]-> (ProgrammingLanguage) : 605 items
   🔸 (Tool) -[:IMPLEMENTS_CONCEPT]-> (Concept) : 2127 items
   🔸 (Tool) -[:USES_LANGUAGE]-> (ProgrammingLanguage) : 220 items
   🔸 (Database) -[:IMPLEMENTS_CONCEPT]-> (Concept) : 368 items
   🔸 (CloudService) -[:IMPLEMENTS_CONCEPT]-> (Concept) : 388 items
   🔸 (Platform) -[:IMPLEMENTS_CONCEPT]-> (Concept) : 400 items
   🔸 (JobRole) -[:REQUIRES_CLOUD_SERVICE]-> (CloudService) : 104 items
   🔸 (Tool) -[:IS_SIMILAR_TO]-> (Tool) : 196 items
   🔸 (Database) -[:IS_SIMILAR_TO]-> (Database) : 54 items
   🔸 (CloudService) -[:IS_SIMILAR_TO]-> (CloudService) : 22 items
   🔸 (Framework) -[:CREATED_BY]-> (Company) : 154 items
   🔸 (Framework) -[:IS_SIMILAR_TO]-> (Framework) : 219 items
   🔸 (Tool) -[:CREATED_BY]-> (Company) : 316 items
   🔸 (Platform) -[:CREATED_BY]-> (Company) : 57 items
   🔸 (ProgrammingLanguage) -[:CREATED_BY]-> (Company) : 65 items
   🔸 (CloudService) -[:CREATED_BY]-> (Company) : 56 items
   🔸 (JobRole) -[:REQUIRES_TOOL]-> (Platform) : 239 items
   🔸 (Database) -[:CREATED_BY]-> (Company) : 26 items
   🔸 (Skill) -[:REQUIRES_CONCEPT]-> (Concept) : 147 items
   🔸 (Tool) -[:BUILT_WITH]-> (ProgrammingLanguage) : 60 items
   🔸 (Platform) -[:BUILT_WITH]-> (ProgrammingLanguage) : 2 items
   🔸 (Platform) -[:IS_SIMILAR_TO]-> (Platform) : 53 items
   🔸 (Concept) -[:IS_SIMILAR_TO]-> (Concept) : 86 items
   🔸 (Concept) -[:IMPLEMENTS_CONCEPT]-> (Concept) : 455 items
   🔸 (ProgrammingLanguage) -[:USES_LANGUAGE]-> (ProgrammingLanguage) : 16 items
   🔸 (ProgrammingLanguage) -[:IS_SIMILAR_TO]-> (ProgrammingLanguage) : 54 items
   🔸 (Concept) -[:null]-> (null) : 12 items
   🔸 (ProgrammingLanguage) -[:IS_SIMILAR_TO]-> (Tool) : 10 items
   🔸 (Tool) -[:IS_SIMILAR_TO]-> (ProgrammingLanguage) : 6 items
   🔸 (Tool) -[:IS_SIMILAR_TO]-> (Platform) : 1 items
   🔸 (Company) -[:LOCATED_IN]-> (Location) : 456 items
   🔸 (ProgrammingLanguage) -[:RELATED_ROLE]-> (ProgrammingLanguage) : 9 items
   🔸 (Platform) -[:REQUIRES_TOOL]-> (ProgrammingLanguage) : 2 items
   🔸 (Tool) -[:REQUIRES_TOOL]-> (Tool) : 39 items
   🔸 (Tool) -[:REQUIRES_TOOL]-> (Platform) : 56 items
   🔸 (Tool) -[:REQUIRES_LANGUAGE]-> (ProgrammingLanguage) : 18 items
   🔸 (ProgrammingLanguage) -[:REQUIRES_TOOL]-> (Tool) : 11 items
   🔸 (ProgrammingLanguage) -[:REQUIRES_TOOL]-> (Platform) : 16 items
   🔸 (Company) -[:CREATED_BY]-> (Tool) : 7 items
   🔸 (ProgrammingLanguage) -[:REQUIRES_LANGUAGE]-> (ProgrammingLanguage) : 1 items
   🔸 (Framework) -[:REQUIRES_LANGUAGE]-> (ProgrammingLanguage) : 27 items
   🔸 (Company) -[:CREATED_BY]-> (ProgrammingLanguage) : 2 items
   🔸 (Company) -[:CREATED_BY]-> (Framework) : 3 items
   🔸 (Platform) -[:REQUIRES_LANGUAGE]-> (ProgrammingLanguage) : 9 items
   🔸 (Concept) -[:CREATED_BY]-> (Company) : 20 items
   🔸 (Concept) -[:REQUIRES_SKILL]-> (Skill) : 13 items
   🔸 (Tool) -[:REQUIRES_SKILL]-> (Skill) : 9 items
   🔸 (Tool) -[:REQUIRES_TOOL]-> (Framework) : 15 items
   🔸 (Platform) -[:REQUIRES_CONCEPT]-> (Concept) : 15 items
   🔸 (Concept) -[:REQUIRES_CONCEPT]-> (Concept) : 118 items
   🔸 (Company) -[:IMPLEMENTS_CONCEPT]-> (Concept) : 47 items
   🔸 (Tool) -[:REQUIRES_CONCEPT]-> (Concept) : 49 items
   🔸 (Company) -[:REQUIRES_TOOL]-> (Tool) : 8 items
   🔸 (Company) -[:REQUIRES_CONCEPT]-> (Concept) : 1 items
   🔸 (Company) -[:REQUIRES_TOOL]-> (Platform) : 2 items
   🔸 (Tool) -[:IS_SIMILAR_TO]-> (Framework) : 3 items
   🔸 (Framework) -[:REQUIRES_TOOL]-> (Tool) : 67 items
   🔸 (Framework) -[:REQUIRES_CONCEPT]-> (Concept) : 55 items
   🔸 (Framework) -[:REQUIRES_TOOL]-> (Platform) : 13 items
   🔸 (Framework) -[:REQUIRES_TOOL]-> (Framework) : 42 items
   🔸 (Framework) -[:USES_TOOL]-> (Tool) : 5 items
   🔸 (Location) -[:LOCATED_IN]-> (Location) : 61 items
   🔸 (CloudService) -[:REQUIRES_TOOL]-> (CloudService) : 18 items
   🔸 (CloudService) -[:REQUIRES_LANGUAGE]-> (ProgrammingLanguage) : 22 items
   🔸 (CloudService) -[:REQUIRES_TOOL]-> (Tool) : 18 items
   🔸 (CloudService) -[:REQUIRES_TOOL]-> (Framework) : 3 items
   🔸 (Concept) -[:USES_LANGUAGE]-> (ProgrammingLanguage) : 4 items
   🔸 (Concept) -[:REQUIRES_TOOL]-> (Tool) : 2 items
   🔸 (Framework) -[:REQUIRES_TOOL]-> (Database) : 9 items
   🔸 (Framework) -[:RELATED_ROLE]-> (Framework) : 2 items
   🔸 (Framework) -[:RELATED_ROLE]-> (JobRole) : 2 items
   🔸 (Platform) -[:USES_LANGUAGE]-> (ProgrammingLanguage) : 47 items
   🔸 (Framework) -[:REQUIRES_PLATFORM]-> (Platform) : 4 items
   🔸 (Tool) -[:REQUIRES_FRAMEWORK]-> (Framework) : 2 items
   🔸 (Company) -[:CREATED_BY]-> (Company) : 14 items
   🔸 (Company) -[:REQUIRES_FRAMEWORK]-> (Framework) : 1 items
   🔸 (Company) -[:REQUIRES_TOOL]-> (Framework) : 2 items
   🔸 (Tool) -[:USES_LANGUAGE]-> (Concept) : 6 items
   🔸 (Framework) -[:USES_LANGUAGE]-> (Concept) : 1 items
   🔸 (Platform) -[:USES_LANGUAGE]-> (Concept) : 1 items
   🔸 (Framework) -[:IS_SIMILAR_TO]-> (Tool) : 3 items
   🔸 (Framework) -[:REQUIRES_TOOL]-> (CloudService) : 10 items
   🔸 (Framework) -[:BUILT_WITH]-> (ProgrammingLanguage) : 6 items
   🔸 (CloudService) -[:USES_LANGUAGE]-> (ProgrammingLanguage) : 12 items
   🔸 (Platform) -[:REQUIRES_TOOL]-> (Tool) : 9 items
   🔸 (Company) -[:CREATED_BY]-> (Platform) : 8 items
   🔸 (Company) -[:CREATED_BY]-> (Database) : 2 items
   🔸 (ProgrammingLanguage) -[:IS_SIMILAR_TO]-> (Framework) : 1 items
   🔸 (Tool) -[:REQUIRES_TOOL]-> (Database) : 17 items
   🔸 (Framework) -[:REQUIRES_CONCEPT]-> (Framework) : 1 items
   🔸 (Framework) -[:REQUIRES_CONCEPT]-> (Platform) : 1 items
   🔸 (Framework) -[:REQUIRES_SKILL]-> (Skill) : 23 items
   🔸 (Framework) -[:IS_SIMILAR_TO]-> (ProgrammingLanguage) : 1 items
   🔸 (Company) -[:CREATED_BY]-> (Concept) : 1 items
   🔸 (Concept) -[:IS_SIMILAR_TO]-> (Platform) : 4 items
   🔸 (Concept) -[:REQUIRES_TOOL]-> (Concept) : 1 items
   🔸 (CloudService) -[:LOCATED_IN]-> (Location) : 11 items
   🔸 (CloudService) -[:CREATED_BY]-> (CloudService) : 1 items
   🔸 (CloudService) -[:REQUIRES_TOOL]-> (Database) : 5 items
   🔸 (CloudService) -[:REQUIRES_CONCEPT]-> (Concept) : 18 items
   🔸 (Tool) -[:CREATED_BY]-> (CloudService) : 3 items
   🔸 (CloudService) -[:REQUIRES_TOOL]-> (Platform) : 4 items
   🔸 (Platform) -[:REQUIRES_TOOL]-> (Database) : 1 items
   🔸 (Tool) -[:REQUIRES_CLOUD_SERVICE]-> (CloudService) : 2 items
   🔸 (Platform) -[:REQUIRES_TOOL]-> (Platform) : 1 items
   🔸 (Location) -[:REQUIRES_CONCEPT]-> (Concept) : 4 items
   🔸 (Concept) -[:REQUIRES_TOOL]-> (Platform) : 1 items
   🔸 (Concept) -[:RELATED_ROLE]-> (Concept) : 15 items
   🔸 (Concept) -[:RELATED_ROLE]-> (Framework) : 1 items
   🔸 (Tool) -[:BUILT_WITH]-> (Tool) : 1 items
   🔸 (Platform) -[:REQUIRES_TOOL]-> (CloudService) : 2 items
   🔸 (Tool) -[:REQUIRES_TOOL]-> (CloudService) : 1 items
   🔸 (JobPosting) -[:IS_FOR_ROLE]-> (JobRole) : 640 items
   🔸 (JobPosting) -[:POSTED_BY]-> (Company) : 634 items
   🔸 (Database) -[:USES_LANGUAGE]-> (ProgrammingLanguage) : 7 items
   🔸 (JobRole) -[:REQUIRES_CONCEPT]-> (ProgrammingLanguage) : 1 items
   🔸 (JobRole) -[:REQUIRES_CLOUD]-> (CloudService) : 13 items
   🔸 (Skill) -[:IMPLEMENTS_CONCEPT]-> (Concept) : 237 items
   🔸 (Database) -[:BUILT_WITH]-> (ProgrammingLanguage) : 2 items
   🔸 (Tool) -[:USES_TOOL]-> (Tool) : 14 items
   🔸 (Platform) -[:USES_LANGUAGE]-> (Tool) : 1 items
   🔸 (Platform) -[:USES_TOOL]-> (Tool) : 1 items
   🔸 (Tool) -[:IS_SIMILAR_TO]-> (Concept) : 7 items
   🔸 (Tool) -[:BUILT_WITH]-> (Framework) : 1 items
   🔸 (JobRole) -[:REQUIRES_TOOL]-> (ProgrammingLanguage) : 2 items
   🔸 (Skill) -[:IMPLEMENTS_CONCEPT]-> (ProgrammingLanguage) : 2 items

🎉 Ingestion Pipeline Complete!


In [ ]:
from neo4j import GraphDatabase
import pandas as pd

# ==========================================
# 1. CONFIGURATION
# ==========================================
class Config:
    NEO4J_URI = "neo4j+s://0c60999b.databases.neo4j.io"
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD = "6Cp_0O2fK3elKHiYIqQsCyo_LiTPQNzobLW_Xw5Z70k"
    DB_NAME = "neo4j" # Standard for AuraDB

# ==========================================
# 2. TEST ENGINE
# ==========================================
def run_test_query(driver, test_name, cypher_query, params=None):
    print(f"\n🔬 RUNNING TEST: {test_name}")
    print("-" * 50)

    try:
        # The modern API calls specific to your request
        records, summary, keys = driver.execute_query(
            cypher_query,
            parameters_=params,  # Use named parameters if needed
            database_=Config.DB_NAME
        )

        # 1. Print Performance Metrics
        print(f"✅ Query executed in {summary.result_available_after} ms")
        print(f"📊 Records returned: {len(records)}")

        # 2. Pretty Print Results (Head 5)
        if records:
            print("\n--- SAMPLE DATA ---")
            # Convert to Pandas DataFrame for cleaner visualization if you have it,
            # otherwise just print the dicts
            data = [r.data() for r in records[:5]]
            for row in data:
                print(row)
        else:
            print("⚠️ No records found (Check your match logic).")

        return records

    except Exception as e:
        print(f"❌ TEST FAILED: {e}")
        return []

# ==========================================
# 3. THE TESTS
# ==========================================
def main():
    with GraphDatabase.driver(Config.NEO4J_URI, auth=(Config.NEO4J_USER, Config.NEO4J_PASSWORD)) as driver:
        driver.verify_connectivity()
        print("🔌 Connected. Starting Verification Suite...")

        # ---------------------------------------------------------
        # TEST 1: The "Reasoning Chain" (Role -> Skill -> Concept)
        # Goal: Verify we didn't just dump keywords, but actually linked HOW to WHY.
        # ---------------------------------------------------------
        query_chain = """
        MATCH path = (role:JobRole)-[:REQUIRES_SKILL]->(skill:Skill)-[:REQUIRES_CONCEPT|IMPLEMENTS_CONCEPT]->(concept:Concept)
        WHERE role.name CONTAINS "Engineer" OR role.name CONTAINS "Data"
        RETURN
            role.name AS Role,
            skill.name AS Required_Skill,
            concept.name AS Underlying_Concept
        LIMIT 5
        """
        run_test_query(driver, "Reasoning Chain Check", query_chain)

        # ---------------------------------------------------------
        # TEST 2: The "Tool Stack" Analysis
        # Goal: Ensure tools are categorized (Framework vs Library vs Database)
        # ---------------------------------------------------------
        query_tools = """
        MATCH (role:JobRole)-[:REQUIRES_TOOL|REQUIRES_FRAMEWORK|REQUIRES_DATABASE]->(tech)
        RETURN
            labels(tech)[0] AS Tech_Type,
            count(*) AS Frequency,
            collect(tech.name)[0..3] AS Examples
        ORDER BY Frequency DESC
        """
        run_test_query(driver, "Tech Stack Categorization", query_tools)

        # ---------------------------------------------------------
        # TEST 3: The "Orphan" Check (Data Quality)
        # Goal: Find nodes that failed to link to anything.
        # (Low numbers are good. High numbers mean extraction failed).
        # ---------------------------------------------------------
        query_orphans = """
        MATCH (n)
        WHERE NOT (n)--()
        RETURN labels(n)[0] AS Entity_Type, count(*) AS Orphan_Count
        ORDER BY Orphan_Count DESC
        """
        run_test_query(driver, "Orphan Node Detection", query_orphans)

if __name__ == "__main__":
    main()

HANDEL FAILED INGESTION

In [ ]:
import json
import re
import os

def audit_failed_ingestion(file_path):
    if not os.path.exists(file_path):
        print(f"❌ Failure log not found at: {file_path}")
        return

    total_failed_triplets = 0
    total_failed_nodes = 0

    print(f"🔍 Auditing Failure Log: {os.path.basename(file_path)}")
    print("-" * 50)

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # 1. Extract JSON arrays from UNWIND statements using Regex
    # This finds everything between UNWIND [ and ] AS
    batches = re.findall(r'UNWIND\s+(\[.*?\])\s+AS', content, re.DOTALL)

    for i, batch_str in enumerate(batches):
        try:
            batch_data = json.loads(batch_str)
            count = len(batch_data)

            # 2. Distinguish between Node batches and Edge batches
            # Node batches are simple lists of strings: ["Python", "Java"]
            # Edge batches are lists of dicts: [{"s": "...", "o": "..."}]
            if count > 0:
                if isinstance(batch_data[0], dict):
                    total_failed_triplets += count
                else:
                    total_failed_nodes += count
        except json.JSONDecodeError:
            print(f"⚠️ Warning: Could not parse batch {i+1} - check for manual file corruption.")

    print(f"📊 AUDIT RESULTS:")
    print(f"   ❌ Failed Nodes:         {total_failed_nodes}")
    print(f"   ❌ Failed Relationships:  {total_failed_triplets}")
    print(f"   📉 Total Missing Items:  {total_failed_nodes + total_failed_triplets}")
    print("-" * 50)

    if total_failed_triplets == 0 and total_failed_nodes == 0:
        print("🎉 The log is empty or contains no valid Cypher blocks. You are likely 100% synced!")

# Run the audit
FAILURE_LOG_PATH = "/content/drive/MyDrive/rr_mlops/failed_ingestion_log.cypher"
audit_failed_ingestion(FAILURE_LOG_PATH)

In [ ]:
import os
from groq import Groq
from google.colab import userdata

api = userdata.get('groq-60days')

client = Groq(
    # This is the default and can be omitted
    api_key=api,
)

models = client.models.list()

print(models)


In [ ]:
def count_total_batches_v2():
    print("🧐 Calculating total batches (80k limit + 1.5k word Wiki cap)...")

    # 1. Initialize Generators with the NEW logic
    role_gen = DataIngestor.read_roles(Config.PATH_ROLES)
    job_gen = DataIngestor.read_jobs_csv(Config.PATH_JOBS)

    # Ensure this matches the 1.5k word cap logic in your DataIngestor
    wiki_gen = DataIngestor.read_wiki(Config.PATH_WIKI)

    # 2. Initialize Batcher with NEW 80,000 character limit
    all_batches = batch_generator(
        [role_gen, job_gen, wiki_gen],
        max_chars=53500
    )

    total_count = 0

    # 3. Exhaust the generator (locally, no API cost)
    for _ in all_batches:
        total_count += 1

    # 4. Results calculation
    # Since you processed 33 batches at 20k, we calculate remaining work
    # based on the new total at 80k.
    # Note: 'processed' is reset because the batch definition changed.
    print("-" * 40)
    print(f"📊 NEW TOTAL BATCHES (80k): {total_count}")
    print(f"⏳ ESTIMATED REMAINING:      ~{total_count} batches")
    print("-" * 40)

    if total_count > 0:
        # Larger batches take longer to process (usually 20-30s including rate limit padding)
        est_seconds_per_batch = 25
        est_minutes = (total_count * est_seconds_per_batch) / 60
        print(f"🕒 Estimated time to finish ALL: ~{round(est_minutes, 1)} minutes.")
        print(f"🚀 Speed Improvement: ~{round(388/total_count, 1)}x faster than 20k batches.")

# Execute the test
count_total_batches_v2()

In [ ]:
def check_source_statistics():
    print("📊 Calculating word counts per source...")

    # Initialize Generators
    sources = {
        "Roles (JSON)": DataIngestor.read_roles(Config.PATH_ROLES),
        "Jobs (CSV)": DataIngestor.read_jobs_csv(Config.PATH_JOBS),
        "Wiki (JSONL)": DataIngestor.read_wiki(Config.PATH_WIKI)
    }

    stats = {}

    for name, gen in sources.items():
        word_count = 0
        chunk_count = 0
        for text in gen:
            # Simple space-based word count
            word_count += len(text.split())
            chunk_count += 1
        stats[name] = {"words": word_count, "chunks": chunk_count}

    print("\n" + "="*40)
    print(f"{'Source':<15} | {'Chunks':<10} | {'Word Count':<15}")
    print("-" * 40)
    total_words = 0
    for name, data in stats.items():
        print(f"{name:<15} | {data['chunks']:<10} | {data['words']:<15,}")
        total_words += data['words']
    print("-" * 40)
    print(f"{'TOTAL':<15} | {'':<10} | {total_words:<15,}")
    print("="*40)

check_source_statistics()